In [17]:
# Description

In [18]:
# -*- coding=utf-8 -*-
"""
주요 기능: Google Drive(API)
    - Google Drive Service 연결
        - GoogleAuth(Super Class) 이용
        - google 사용자 계정(user_nick), google 프로젝트 봇 계정(bot_nick)에 적용 가능
    - Google Drive 상용 기능 함수화(Find/Create/Updata/Delete/Upload/Download)
        - 단일 계정 내부 함수
            - list: GoogleAPI(super class) 상속
            - read: file 내용(Non-Google 파일)
            - write: file 변경(Non-Google 파일)
            - create: create_file / create_folder
            - update: update_file / update_folder
            - delete: delete_file / delete_folder
            - copy: 파일 복사
            - convert_mime_type: mime_type 변경(RestAPI로 구현)  ## TODO: drive service(v3)로 구현가능한 방법 모색
            - move: 소스 폴더의 해당파일 목적 폴더로 이동
            - move_all: 소스 폴더의 모든 파일을 목적 폴더로 이동

        - 복수 계정간 함수
            - move_to
            - copy_to
            - _copy_to_colab
            - _copy_to_sheets

    - 로컬 드라이브에 적용
        - 로컬: Jupyter Server 실행
        - Colab: 로컬 런타임에 연결

사용 방법:
    ```
    gds = GoogleDrive(user_nick="master", path="/__COLAB/_TEST")
    gds.list(type="file")
    ```

참조 자료:
    - 

필요 환경:
    - 인증 관련 json 파일
    - super class: Google.Google
    - requirements: 
    - python:
"""

'\n주요 기능: Google Drive(API)\n    - Google Drive Service 연결\n        - GoogleAuth(Super Class) 이용\n        - google 사용자 계정(user_nick), google 프로젝트 봇 계정(bot_nick)에 적용 가능\n    - Google Drive 상용 기능 함수화(Find/Create/Updata/Delete/Upload/Download)\n        - 단일 계정 내부 함수\n            - list: GoogleAPI(super class) 상속\n            - read: file 내용(Non-Google 파일)\n            - write: file 변경(Non-Google 파일)\n            - create: create_file / create_folder\n            - update: update_file / update_folder\n            - delete: delete_file / delete_folder\n            - copy: 파일 복사\n            - convert_mime_type: mime_type 변경(RestAPI로 구현)  ## TODO: drive service(v3)로 구현가능한 방법 모색\n            - move: 소스 폴더의 해당파일 목적 폴더로 이동\n            - move_all: 소스 폴더의 모든 파일을 목적 폴더로 이동\n\n        - 복수 계정간 함수\n            - move_to\n            - copy_to\n            - _copy_to_colab\n            - _copy_to_sheets\n\n    - 로컬 드라이브에 적용\n        - 로컬: Jupyter Server 실행\n        - Colab: 로컬 런타임에 연결\n\n사용 방법:\n   

# References
- [Google Drive for Developers > Drive API(V3)](https://developers.google.com/drive/api/v3/reference)
- [Google Drive for Developers > Drive API(V3) > Files](https://developers.google.com/drive/api/v3/reference/files)

# Import Packages

## BuiltIn Packages

In [19]:
##@@@ Package/Module
##============================================================

##@@ Built-In Package/Module
##------------------------------------------------------------
import os
import sys
import io
import json

from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from googleapiclient.http import MediaIoBaseUpload

## External Packages(installed by pip)

In [20]:
## Local(PC Drive) 환경인지 Cloud(Google Drive) 환경인지 확인
ENV = "_LOCAL_"
if "content" in os.getcwd():
    ENV = "_GDRIVE_"

if ENV == "_LOCAL_":
    try: # LOCAL SHELL
        cwd = __file__
    except:  # LODAL JUPYTER
        import import_ipynb  ## NOTE: .ipynb을 import 할 수 있도록 하는 모듈
else:
    sys.path.insert(0, '/content/drive/MyDrive/__COLAB/PACKAGES')
    import import_ipynb  ## NOTE: .ipynb을 import 할 수 있도록 하는 모듈
    sys.path.insert(0, '/content/drive/MyDrive/__COLAB/__GOOGLE')
    ## NOTE: 현재 디렉토리로 이동한 후 import해야 함
    # %cd /content/drive/MyDrive/__COLAB/__GOOGLE

from GoogleAPI import GoogleAPI
PATH_CONFIGS = "./configs"

# Global Constants

In [21]:


##@@@ Constant/Varible
##============================================================

##@@ Constant
##------------------------------------------------------------


##@@ Variable(Golobal)
##------------------------------------------------------------


# Private Functions

## Section1

In [22]:
##@@@ Private Function
##============================================================


##@@ sub function
##------------------------------------------------------------

# Main Class

In [23]:
##@@@ Main Class
##============================================================
class GoogleDrive(GoogleAPI):
    def __init__(self, user_nick=None, bot_nick=None, path=None):
        """__init__: Object 초기화
        Desc:
            - Google Drive Service 연결(user_nick)
            - path 폴더 기준

        Args:
            - user_nick(str, None): 구글계정 별칭(master: monblue@snu.ac.kr / moonitdev: moonitdev@gmail.com / ...)
            - bot_nick(str, None): GCP(google cloud platform) Project 서비스 계정(Bot) 별칭(mats, moonMaster, ...)
            - path(str, None): Google Drive 기준 폴더(경로) 예) __COLAB/_TEST

        Usages:
            - [description]
        """
        super().__init__("drive", user_nick=user_nick, bot_nick=bot_nick)
        # self.service = build("drive", self.version["drive"], credentials = self.credentials)
        self._set_id(path)


    ##@@ Private Function
    ##------------------------------------------------------------

    ##@ Inner Google Drive
    ##------------------------------------------------------------
    def _meta_data(self, file_id, fields="id, name, parents, mimeType"):
        return self.service.files().get(fileId=file_id, fields=fields).execute()


    def _root_folder_id(self):
        return "root"


    def _set_id(self, path, auto_create=True):
        self.path = path
        self.id = self._root_folder_id()  # self.id 초기화
        paths = self._ids_by_path(path=path, folder=False)  # GoogleAPI에서 상속
        id = list(paths[-1].values())[0]  # 마지막 path 항목 id
        if auto_create:  # 폴더 자동 생성
            id = self._create_folders_by_paths(paths)

        self.id = id if id != None else self.id


    def _read(self, file_id, mime_type=None):
        # # TODO: io.BytesIO 사용해야 하는 경우 찾을 것!!
        # if type == 'doc' or 'doc' in path[-6:]:  ## NOTE: doc, docx 파일은 별도 type 처리
        #     print(f"fileId: {file_id}, path: {path}, type: {type}")
        #     request = self.service.files().export_media(fileId=file_id, mimeType='text/csv')
        #     fh = io.BytesIO(path)
        # else:  ## NOTE: type = binary / text 의미 없음
        #     request = self.service.files().get_media(fileId=file_id)
        #     fh = open(path, 'wb')
        # media = self.service.files().get_media(fileId=file_id).execute()
        try:
            media = self.service.files().get_media(fileId=file_id).execute()
        except HttpError as e: # googleapiclient.errors.HttpError
            # print(e)
            # mime_type = self._meta_data(file_id)['mimeType']
            # mime_type = _export_mime_type(mime_type)  ## TODO
            mime_type = "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
            media = self.service.files().export_media(fileId=file_id, mimeType=mime_type).execute()

        return media
        # return self.service.files().get_media(fileId=file_id).execute()


    def _copy(self, file_id, name=None):
        # https://developers.google.com/drive/api/v3/reference/files/copy
        # name: 파일 이름
        # parents: 목적 폴더
        # NOTE: mimeType: mime_type 변경시 <- type="colab"
        # if 'type' in dst_meta_data:
        #     dst_meta_data['mimeType'] = self.mime_type[dst_meta_data.pop('type')]
        return self.service.files().copy(
            fileId = file_id,
            name = name,
        ).execute()


    def _move(self, file_id, src_folder_id, dst_folder_id):
        return self.service.files().update(
                fileId = file_id,
                addParents = dst_folder_id,
                removeParents = src_folder_id
        ).execute()


    def _move_all(self, src_folder_id, dst_folder_id):
        query = f"parents = '{src_folder_id}' and trashed = false"
        response = self.service.files().list(q=query).execute()
        files = response.get('files')
        nextPageToken = response.get('nextPageToken')

        while nextPageToken:
            response = self.service.files().list(q=query, pageToken=nextPageToken).execute()
            files.extend(response.get('files'))
            nextPageToken = response.get('nextPageToken')
        
        for f in files:
            if f['mimeType'] != 'application/vnd/google-apps.folder':  ## NOTE: folder가 아닌 파일만
                self._move(f.get('id'), src_folder_id, dst_folder_id)


    def _write(self, file_id, data=None, meta_data={}, in_type="text"):
        _meta_data = self._meta_data(file_id=file_id, fields="name, mimeType, parents")
        parents = _meta_data.pop("parents")
        print(f"data: {data}, _meta_data: {_meta_data}")
        # _meta_data["fileId"] = file_id

        if data and _meta_data:
            meta_data = dict(_meta_data, **meta_data)
            mime_type = meta_data["mimeType"]
            if in_type[0].lower() == "t": ## 
                media_body = MediaIoBaseUpload(io.BytesIO(json.dumps(data).encode('utf-8')), mime_type, resumable=True)  ## NOTE: string(text)
            elif in_type[0].lower() == "b":  ## binary data
                media_body = MediaIoBaseUpload(io.BytesIO(data), mime_type, resumable=True)  ## NOTE: Google Drive File
            elif in_type[0].lower() == "g" or in_type[0].lower() == "g":  ## google drive file_id
                media_body = MediaIoBaseUpload(io.BytesIO(self.drive.files().get_media(fileId=data).execute()), mime_type, resumable=True)  ## NOTE: Google Drive File
            else:  ## local drive file_path
                media_body = MediaFileUpload(data, mimetype=mime_type, resumable=True)  ## NOTE: Local File

            ## update file
            return self.drive.files().update(
                fileId = file_id,
                body = meta_data,
                media_body = media_body
            ).execute()

            ## create file
            # return self.drive.files().create(
            #     fileId = file_id,
            #     body = meta_data,
            #     media_body = media_body
            # ).execute()

        else:
            print("해당 파일이 없습니다.")
            return None


    def _delete_file_by_file_id(self, file_id):
        """파일 삭제(file id)

        Args:
            file_id (str): 삭제할 파일의 file_id
        """
        return self.service.files().delete(fileId=file_id).execute()


    def _share_file(self, file_id, user_permission, domain_permission):
        def callback(request_id, response, exception):
            if exception:
                # Handle error
                print(exception)
            else:
                print("Permission Id: %s" % response.get('id'))

        batch = self.service.new_batch_http_request(callback=callback)
        user_permission = {
            'type': 'user',
            'role': 'writer',
            # 'role': 'owner',
            'emailAddress': 'moonitdev@gmail.com'
            # 'emailAddress': 'monblue@snu.ac.kr'
        }
        batch.add(self.service.permissions().create(
            fileId = file_id,
            body = user_permission,
            fields = 'id',
        ))
        domain_permission = {
            'type': 'domain',
            'role': 'reader',
            'domain': 'example.com'
        }
        batch.add(self.service.permissions().create(
            fileId = file_id,
            body = domain_permission,
            fields='id',
        ))
        batch.execute()


    ##@ Google Drive - Local Drive
    ##------------------------------------------------------------

    # def _upload_file(self, path, name=None, folder_path=None, mime_type=None):
    def _upload(self, src_path, meta_data={}):
        """파일 업로드

        Args:
            src_path (str): 업로드할 파일 경로
            meta_data (dict): 필수: {"name": "", "parents": ""}  # 선택: mimeType, 

        Returns:
            [dict]: {<file_name>: <file_id>}
        """
        meta_data = dict(meta_data, resumable=True)
        media = MediaFileUpload(src_path, **meta_data)

        return self.service.files().create(
            body = metadata,
            media_body = media,
            fields = 'id'
        ).execute()['id']


    def _download(self, file_id, dst_path=None, mode='wb'):
        """파일 다운로드(file_id)

        Args:
            file_id (str): 다운로드할 파일의 file_id
            dst_path (str): 저장할 경로
            type (str): 다운로드할 파일 타입('binary': 이진 파일 / 'text': 텍스트 파일)
        """

        mode = mode if type[0].lower() == 'w' else 'wb' if type[0].lower() == 'b' else 'wt'
        # request = self.service.files().get_media(fileId=file_id)
        data = self._read(file_id)
        fh = open(dst_path, mode)

        downloader = MediaIoBaseDownload(fh, data)

        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}")


    ##@ Google Drive1 - Google Drive2
    ##------------------------------------------------------------


    ##@@ Public Function
    ##------------------------------------------------------------
    def list(self): # GoogleAPI에서 상속
        pass


    def create_folder(self, path):
        (parents_path, name) = path.rsplit("/", 1)
        parent_id = self.id_by_path(parents_path)  # GoogleAPI에서 상속
        return self._create_folder_by_parent_id(name, parent_id)


    def change_folder(self, path):
        self._set_id(path)


    def read(self, file_path):
        return self._read(self._id_by_path(file_path))


    def move(self, file_id, src_folder_path, dst_folder_path):
        return self._move(file_id, self._id_by_path(src_folder_path, auto_create=False), self._id_by_path(dst_folder_path, auto_create=True))


    def move_all(self, src_folder_path, dst_folder_path):
        self._move_all(self._id_by_path(src_folder_path, auto_create=False), self._id_by_path(dst_folder_path, auto_create=True))


    def delete(self, path):
        return self._delete_file_by_id(self.id_by_path(path))


    # def convert_mime_type(self, file_id, type="colab"):
    #     """convert_type: file mime_type 변경
    #     Desc:
    #         - NOTE: Google 편집 파일로의 변경만 가능
    #     """
    #     self.copy(file_id=file_id, type=type)
    #     self.delete(file_id=file_id)


    # def _convert_excel_file(self, file_path='', folder_ids=[]):
    #     if not os.path.exists(file_path):
    #         print(f"{file_path} not found")
    #         return
        
    #     try:
    #         file_metadata = {
    #             'name': os.path.splitext(os.path.basename(file_path))[0],
    #             'mimeType': "application/vnd.google-apps.spreadsheet",
    #             'parents': folder_ids
    #         }

    #         media =  MediaFileUpload(
    #             filename=file_path, 
    #             mime_type="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet", 
    #         )

    #         response = self.service.files().create(
    #             media_body = media,
    #             body = file_metadata
    #         ).execute()

    #         print(response)
    #         return response
    #     except Exception as e:
    #         print(e)
    #         return

    ##
    ##----------------------------------------------------------------------

    ##@ Google Drive - Local Drive
    ##------------------------------------------------------------
    def upload(self, src_path, dst_path):
        meta_data = {}
        (meta_data['name'], meta_data['parents']) = self._name_parents_by_path(dst_path, auto_create=True)

        return self._upload(src_path, meta_data)


    def download(self, src_path, dst_path):
        """파일 다운로드

        Args:
            name (str): google drive 내의 파일 이름
            folder (str, optional): 저장할 폴더 이름. Defaults to '.'
        """
        self._download(self.id_by_name(src_path), path=dst_path)


    ##@ Google Drive1 - Google Drive2
    ##------------------------------------------------------------



# main(`__init__`)

In [24]:
##@@@ Main Function
##============================================================
if __name__ == "__main__":
    pass
    user_nick = "satsbymoon"
    path = "/__COLAB/__GOOGLE"

    ##@@ NOTE: drive
    gds = GoogleDrive(user_nick=user_nick, bot_nick=None, path=path)
    file_id = "1sbIbVODJniXPfI4-i7eoMKavWoFcobBZ"  # json 파일
    file_id = "1tMLK_aYO5f2SIYgBYE7pDD64nfDJWfzGcsOtEVOiCwQ"  # google sheets 파일
    file_id = "1wXyfsi7uGLIXTVPplhfUWwHTAr4st-AL"  # text 파일 /__COLAB/__GOOGLE/test.txt
    file_id = "1Nd5QRWfXCdS8UrATBb-WnSjTLH-Uj2P0"  # text 파일 /__COLAB/__GOOGLE/test.txt
    # r = gds._read(file_id, mime_type=None)
    data = "write test... ReWrite22"
    r = gds._write(file_id, data=data, meta_data={}, in_type="text")
    print(r)

paths: [{'': None}, {'__COLAB': None}, {'__GOOGLE': None}]
data: write test... ReWrite22, _meta_data: {'name': 'test.txt', 'mimeType': 'text/plain'}
{'kind': 'drive#file', 'id': '1Nd5QRWfXCdS8UrATBb-WnSjTLH-Uj2P0', 'name': 'test.txt', 'mimeType': 'text/plain'}


## list

In [25]:
    # file_id = '1wXyfsi7uGLIXTVPplhfUWwHTAr4st-AL'
    # file_id = '1d2HmMAYmgtsv6e5ugxW696JExBmvAYAp_-jxioY2qLw'  ## test_
    # l = gds.list(queries=["parents = '1ywx8F3Ujx0wH093wznXf0qgYo1mhKDRs'"], trashed=False, out_type="list")
    # print(l)
    # src_folder_id = "1ywx8F3Ujx0wH093wznXf0qgYo1mhKDRs"
    # dst_folder_id = "1cXwU3nf9_CldBFWzb8DzJqlgs3sDyc7P"
    # gds._move(file_id, src_folder_id=src_folder_id, dst_folder_id=dst_folder_id)

    # src_folder_path = "/__COLAB/__GOOGLE"
    # dst_folder_path = "/__COLAB/_TEST"
    # gds.move(file_id, src_folder_path=src_folder_path, dst_folder_path=dst_folder_path)

    # src_folder_id = "1IBfGzsJMaFn0gYes6iJGmgka36v1aeHu"
    # dst_folder_id = "1H4A_4hyRHJUvyDkuCcWhXvWUYe_4PMQz"
    # gds._move_all(src_folder_id=src_folder_id, dst_folder_id=dst_folder_id)

    # src_folder_path = "/__COLAB/_TEST/TEST2"
    # dst_folder_path = "/__COLAB/_TEST/TEST1"
    # gds.move_all(src_folder_path=src_folder_path, dst_folder_path=dst_folder_path)


    # gds._copy(file_id, name="test_2")
    # gds._copy(file_id, name="test_2", parents=["1H4A_4hyRHJUvyDkuCcWhXvWUYe_4PMQz", "1IBfGzsJMaFn0gYes6iJGmgka36v1aeHu"])
    # title
    
    # print(l)
    # id = gds.upload_file("_Google.py", "_GoogleDrive.py")
    # print(gds.service.files().get(fileId=id).execute())

    # 1wXyfsi7uGLIXTVPplhfUWwHTAr4st-AL

## Section2

# PlayGround